In [1]:
import pandas as pd
import numpy as np
import duckdb
import duckdb_deps # duckdb dependencies


In [2]:
# initialize duckdb
conn, cursor = duckdb_deps.init_duckdb()

In [3]:
# Create the index
cursor = conn.cursor()
cursor.execute("CREATE INDEX academic_article_index ON author_article (academic_id, article_id)")
cursor.execute("CREATE INDEX booktitle_improceeding_idx ON booktitle_improceeding (booktitle_id)")
cursor.execute("CREATE INDEX improceeding_idx ON improceeding (improceeding_id)")

In [4]:
# This query gives back the publisher(s) that had as proceeding_title 'PODS' in it. The NaN has been excluded. 
queryE1 = """
            SELECT publisher.publisher_name 
            FROM publisher 
            JOIN publisher_proceeding ON publisher.publisher_id=publisher_proceeding.publisher_id 
            JOIN proceeding ON proceeding.proceeding_id=publisher_proceeding.proceeding_id 
            WHERE proceeding.proceeding_title LIKE '%PODS%' AND publisher.publisher_name IS NOT NULL
            GROUP BY publisher_name
            """

E1 = cursor.execute(queryE1).fetchdf()
E1

,publisher_name
0,ACM
1,CWI


In [7]:
# This query gives back the articles that have been published in the journal 'Theory Comput. Syst.' and written by 'Martin Grohe' 
queryE2 = """
            SELECT article.article_title 
            FROM article JOIN author_article ON author_article.article_id=article.article_id 
            JOIN academic ON academic.academic_id=author_article.academic_id 
            JOIN journal ON article.journal_id=journal.journal_id 
            WHERE journal.journal_name='Theory Comput. Syst.' 
            AND academic.academic_name='Martin Grohe' 
            ORDER BY article.article_title
            """

E2 = cursor.execute(queryE2).fetchdf()
E2

,article_title
0,Database Query Processing Using Finite Cursor ...
1,Learnability and Definability in Trees and Sim...
2,Tight Lower and Upper Bounds for the Complexit...


In [8]:
# This query is going the GROUP BY year to find the number of published articles for each year,
# this if filtered to only count the articles with as booktlte_name having 'sigmod' in the name and in the year 2022
# So we get back the number of articlees published in 2022 with as booktitle like 'sigmod'. 
queryM1 = """
            SELECT COUNT(year) as number_of_articles_published, year 
            FROM booktitle 
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id 
            JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id 
            WHERE lower(booktitle.booktitle_name) LIKE '%sigmod%' 
            AND year='2022' 
            GROUP BY year
            """

M1 = cursor.execute(queryM1).fetchdf()
M1





,number_of_articles_published,year
0,282,2022


In [14]:
queryM2 = """
SELECT journal_name, COUNT(journal_name) as num_articles
FROM article, journal
where article.journal_id = journal.journal_id
GROUP BY journal_name
ORDER BY num_articles DESC
LIMIT 1;
"""
M2 = cursor.execute(queryM2).fetchdf()
M2

,journal_name,num_articles
0,CoRR,457480


In [15]:
# Execute the query
# This is a subquery that is going to list all the improceedings published in each year using COUNT, GROUP BY and WHERE
# to filter only the booktitle's with as name 'CIDR' 
queryM3_1 = """
            SELECT COUNT(improceeding.improceeding_id) as frequency, improceeding.year AS improceedings_per_year
            FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id = booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id = booktitle_improceeding.improceeding_id
            WHERE booktitle.booktitle_name LIKE '%CIDR%'
            GROUP BY improceeding.year
            ORDER BY improceeding.year
            """

# This query uses the PERCENTILE_CONT function, but the MEDIAN function would also be used.
# With the list given above it is simply going to take the median of it using the function just cited. 
queryM3 = """
            SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY frequency) as median_frequency
            FROM (
            SELECT count(improceeding.improceeding_id) as frequency, improceeding.year AS improceedings_per_year
            FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id = booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id = booktitle_improceeding.improceeding_id
            WHERE booktitle.booktitle_name = 'CIDR'
            GROUP BY improceeding.year
            ORDER BY improceeding.year 
            )
            """

M3 = cursor.execute(queryM3).fetchdf()
M3

,median_frequency
0,45.0


In [16]:
# This subquery is going to give back all improceedings that have more than 10 authors. 
queryM4_1 = """
            SELECT * FROM 
            (SELECT COUNT(improceeding.improceeding_id) as improceeding_authors, improceeding.improceeding_id 
            FROM author_inproceeding 
            JOIN improceeding ON improceeding.improceeding_id=author_inproceeding.improceeding_id 
            GROUP BY improceeding.improceeding_id 
            ORDER BY improceeding_authors) x 
            WHERE x.improceeding_authors > 10
            """

# This subquery is going to give back a list of the number of improceedings for each year, same as queryM3_1. 
# But there is an additional layer (using (sub)queryM4_1) to filter to improceedings with at least 10 authors. 
queryM4_2 = """
        SELECT * 
            FROM 
                (SELECT COUNT(year) as improceeding_frequency_10, year 
                FROM 
                    (SELECT improceeding_id 
                    FROM 
                        (SELECT COUNT(improceeding.improceeding_id) as improceeding_authors, improceeding.improceeding_id 
                        FROM author_inproceeding 
                        JOIN improceeding ON improceeding.improceeding_id=author_inproceeding.improceeding_id 
                        GROUP BY improceeding.improceeding_id 
                        ORDER BY improceeding_authors) x 
                        WHERE x.improceeding_authors > 10) y 
                        JOIN improceeding ON improceeding.improceeding_id=y.improceeding_id 
                        JOIN booktitle_improceeding ON booktitle_improceeding.improceeding_id=improceeding.improceeding_id 
                        JOIN booktitle ON booktitle.booktitle_id=booktitle_improceeding.booktitle_id 
                        WHERE booktitle.booktitle_name LIKE '%SIGMOD%' 
                        GROUP BY year 
                        ORDER BY improceeding_frequency_10 DESC)"""
 
# This query is simply going to take the max of improceedings from the list made by (sub)queryM4_2, 
# it is also going to give in which the year it was. 
queryM4 = """
            SELECT MAX(improceeding_frequency_10) as number_of_SIGMOD_conference_higher_than_10_authors, ANY_VALUE(year) as year 
            FROM 
                (SELECT COUNT(year) as improceeding_frequency_10, year 
                FROM 
                    (SELECT improceeding_id 
                    FROM 
                        (SELECT COUNT(improceeding.improceeding_id) as improceeding_authors, improceeding.improceeding_id 
                        FROM author_inproceeding 
                        JOIN improceeding ON improceeding.improceeding_id=author_inproceeding.improceeding_id 
                        GROUP BY improceeding.improceeding_id 
                        ORDER BY improceeding_authors) x 
                        WHERE x.improceeding_authors > 10) y 
                        JOIN improceeding ON improceeding.improceeding_id=y.improceeding_id 
                        JOIN booktitle_improceeding ON booktitle_improceeding.improceeding_id=improceeding.improceeding_id 
                        JOIN booktitle ON booktitle.booktitle_id=booktitle_improceeding.booktitle_id 
                        WHERE booktitle.booktitle_name LIKE '%SIGMOD%' 
                        GROUP BY year 
                        ORDER BY improceeding_frequency_10 DESC)"""

M4 = cursor.execute(queryM4).fetchdf()
M4

,number_of_SIGMOD_conference_higher_than_10_authors,year
0,13,2020


In [17]:
# This subquery is going to give the list of the editors and the number of times they published in the PODS conferences. 
queryM5_1 = """
            SELECT COUNT(academic.academic_id) AS author_frequency, ANY_VALUE(academic.academic_name)
            FROM academic 
            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id 
            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id 
            JOIN booktitle_proceeding ON proceeding.proceeding_id=booktitle_proceeding.proceeding_id
            JOIN booktitle ON booktitle.booktitle_id=booktitle_proceeding.booktitle_id
            WHERE booktitle.booktitle_name LIKE '%PODS%' 
            GROUP BY academic.academic_id
            ORDER BY author_frequency DESC"""

# This query is going to use the subquery M5_1 to find the list from above. 
# Now the goal is to give back everytime all the editors that have the same maximum number. 
# This cannot be done with a function like LIMIT or OFFSET which would give a hardcoded top list. 
# So this has been done so that it gives back all the editors with the same (maximum) frequencies. 
# This maximum is taking from another subquery. 
queryM5 = """
            SELECT academic_name, author_frequency FROM
            (SELECT COUNT(academic.academic_name) AS author_frequency, academic.academic_name 
            FROM academic 
            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id 
            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id 
            JOIN booktitle_proceeding ON proceeding.proceeding_id=booktitle_proceeding.proceeding_id
            JOIN booktitle ON booktitle.booktitle_id=booktitle_proceeding.booktitle_id
            WHERE booktitle.booktitle_name LIKE '%PODS%' 
            GROUP BY academic.academic_name)
            WHERE author_frequency = (SELECT MAX(author_frequency) FROM     (SELECT COUNT(academic.academic_name) AS author_frequency, academic.academic_name 
                                                                            FROM academic 
                                                                            JOIN editor_proceeding ON academic.academic_id=editor_proceeding.academic_id 
                                                                            JOIN proceeding ON proceeding.proceeding_id=editor_proceeding.proceeding_id 
                                                                            JOIN booktitle_proceeding ON proceeding.proceeding_id=booktitle_proceeding.proceeding_id
                                                                            JOIN booktitle ON booktitle.booktitle_id=booktitle_proceeding.booktitle_id
                                                                            WHERE booktitle.booktitle_name LIKE '%PODS%' 
                                                                            GROUP BY academic.academic_name))"""

M5 = cursor.execute(queryM5_1).fetchdf()
M5

,author_frequency,any_value(academic.academic_name)
0,3,Maurizio Lenzerini
1,3,Leonid Libkin
2,3,Josep Lluís Larriba-Pey
3,2,Georgia Koutrika
4,2,George H. L. Fletcher
5,2,Mirek Riedewald
6,2,Akhil Arora 0001
7,2,Arnab Bhattacharya 0001
8,2,Kostas Stefanidis
9,2,Laks V. S. Lakshmanan


In [18]:
# This subquery finds the academic with the most publishings (in conferences & journals combined), 
# and give the academic_id back (together with the amount of publishings)
queryM6_1 = """
            SELECT MAX(Z.total_publishings), ANY_VALUE(Z.academic_id)
            FROM
            (SELECT X.academic_id, number_of_improceedings, number_of_journal_articles, number_of_improceedings+number_of_journal_articles as total_publishings
            FROM 
                (SELECT COUNT(author_inproceeding.academic_id) as number_of_improceedings, author_inproceeding.academic_id 
                FROM academic 
                JOIN author_inproceeding ON author_inproceeding.academic_id=academic.academic_id 
                GROUP BY author_inproceeding.academic_id) X 
                JOIN 
                    (SELECT COUNT(author_article.academic_id) as number_of_journal_articles, author_article.academic_id 
                    FROM academic 
                    JOIN author_article ON author_article.academic_id=academic.academic_id 
                    GROUP BY author_article.academic_id) Y 
                ON X.academic_id=Y.academic_id ORDER BY total_publishings DESC) Z
            WHERE NOT(Z.academic_id=-1)"""

# This query gives there number of different conferences published (in this case 90) given an academic_id (here hardcoded as '968212')
queryM6_2 = """
            SELECT COUNT(booktitle_name) FROM
            (SELECT COUNT(booktitle.booktitle_name), booktitle.booktitle_name FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id
            JOIN author_inproceeding ON author_inproceeding.improceeding_id=improceeding.improceeding_id
            JOIN academic ON academic.academic_id=author_inproceeding.academic_id
            WHERE academic.academic_id=968212
            GROUP BY booktitle.booktitle_name)"""

# Now for this query, we combine the two precedent queries together, subqueryM6_1 to find the academic_id 
# and subqueryM6_2 to find the number of different conferences that academic has published. 
queryM6 = """
            SELECT COUNT(booktitle_name) AS number_of_different_conferences_published FROM
            (SELECT COUNT(booktitle.booktitle_name), booktitle.booktitle_name FROM booktitle
            JOIN booktitle_improceeding ON booktitle_improceeding.booktitle_id=booktitle.booktitle_id
            JOIN improceeding ON improceeding.improceeding_id=booktitle_improceeding.improceeding_id
            JOIN author_inproceeding ON author_inproceeding.improceeding_id=improceeding.improceeding_id
            JOIN academic ON academic.academic_id=author_inproceeding.academic_id
            WHERE academic.academic_id=(SELECT A.academic_id
                                        FROM
                                        (SELECT MAX(Z.total_publishings), ANY_VALUE(Z.academic_id) AS academic_id
                                        FROM
                                        (SELECT X.academic_id, number_of_improceedings, number_of_journal_articles, number_of_improceedings+number_of_journal_articles as total_publishings
                                        FROM 
                                            (SELECT COUNT(author_inproceeding.academic_id) as number_of_improceedings, author_inproceeding.academic_id 
                                            FROM academic 
                                            JOIN author_inproceeding ON author_inproceeding.academic_id=academic.academic_id 
                                            GROUP BY author_inproceeding.academic_id) X 
                                            JOIN 
                                                (SELECT COUNT(author_article.academic_id) as number_of_journal_articles, author_article.academic_id 
                                                FROM academic 
                                                JOIN author_article ON author_article.academic_id=academic.academic_id 
                                                GROUP BY author_article.academic_id) Y 
                                            ON X.academic_id=Y.academic_id ORDER BY total_publishings DESC) Z
                                        WHERE NOT(Z.academic_id=-1)) A)
            GROUP BY booktitle.booktitle_name)
            """

M6 = cursor.execute(queryM6).fetchdf()
M6

,number_of_different_conferences_published
0,90


In [19]:
# This query gives back the authors of academic_id '168668' in a list and the number of times they published with them.
# A LIMIT 1 could be used to just keep the most frequent author (or one of them).  
queryH1_1 = """
                SELECT academic2.academic_name, COUNT(*) as coauthor_count
                FROM author_article author_article1
                JOIN author_article author_article2 ON author_article1.article_id = author_article2.article_id
                JOIN academic academic1 ON author_article1.academic_id = academic1.academic_id
                JOIN academic academic2 ON author_article2.academic_id = academic2.academic_id
                WHERE academic1.academic_id = '168668' AND academic2.academic_id != '168668'
                GROUP BY academic2.academic_name
                ORDER BY coauthor_count DESC
                """

# The goal for the query is to do the same as the previous query but doing it for every academic_id. 
queryH1 = """
                SELECT ANY_VALUE(academic1.academic_id), ANY_VALUE(academic1.academic_name), ANY_VALUE(academic2.academic_name) as coauthor, COUNT(*) as coauthor_count
                FROM author_article author_article1
                JOIN author_article author_article2 ON author_article1.article_id = author_article2.article_id
                JOIN academic academic1 ON author_article1.academic_id = academic1.academic_id
                JOIN academic academic2 ON author_article2.academic_id = academic2.academic_id
                WHERE academic2.academic_id != academic1.academic_id
                GROUP BY academic1.academic_id, academic2.academic_name
                ORDER BY academic1.academic_id, coauthor_count DESC
            """

H1 = cursor.execute(queryH1_1).fetchdf()
H1

,academic_name,coauthor_count
0,Donatella Firmani,3
1,Paolo Merialdo,3
2,Denilson Barbosa,2
3,Antonio Matinata,2
4,Tommaso Teofili,1


In [20]:
# This query will look up if the two academics given are co-author of an improceeding, if they are the query will give 0 back, otherwise 1. 
queryH2_1 = """
            SELECT 
            CASE 
                WHEN academic1.academic_id IN (SELECT academic_id 
                                        FROM author_inproceeding 
                                        WHERE improceeding_id IN 
                                            (SELECT improceeding_id 
                                            FROM author_inproceeding 
                                            WHERE academic_id = academic2.academic_id)) 
                                        THEN 0
                ELSE 1
            END AS distance
            FROM academic academic1
            JOIN academic academic2 ON academic1.academic_id <> academic2.academic_id
            WHERE academic1.academic_name = 'Maurizio Lenzerini' AND academic2.academic_name = 'Martin Grohe'
            """

# This query should count the distance between two given academics. By using "WITH RECURSIVE" and applying the shortest past algorithm. 
queryH2 = """
            WITH RECURSIVE co_publishers AS (
              SELECT academic1.academic_id, academic2.academic_id AS co_publisher_id, 0 AS distance
              FROM academic academic1
              JOIN author_inproceeding author_inproceeding1 ON academic1.academic_id = author_inproceeding1.academic_id
              JOIN author_inproceeding author_inproceeding2 ON author_inproceeding1.improceeding_id = author_inproceeding2.improceeding_id AND author_inproceeding2.academic_id <> academic1.academic_id
              JOIN academic academic2 ON author_inproceeding2.academic_id = academic2.academic_id
              WHERE academic1.academic_name = 'Maurizio Lenzerini'
              UNION ALL
              SELECT co_publishers.academic_id, academic3.academic_id AS co_publisher_id, co_publishers.distance + 1 AS distance
              FROM co_publishers
              JOIN author_inproceeding author_inproceeding1 ON co_publishers.co_publisher_id = author_inproceeding1.academic_id
              JOIN author_inproceeding author_inproceeding2 ON author_inproceeding1.improceeding_id = author_inproceeding2.improceeding_id AND author_inproceeding2.academic_id <> co_publishers.co_publisher_id
              JOIN academic academic3 ON author_inproceeding2.academic_id = academic3.academic_id
            )
            SELECT MIN(distance) AS distance
            FROM co_publishers
            WHERE co_publisher_id IN (SELECT academic_id FROM academic WHERE academic_name = 'Martin Grohe')
            """

H2 = cursor.execute(queryH2).fetchdf()
H2

In [ ]:
# The query returns the names of all pairs of academics who have edited the same proceedings, 
# but excludes pairs where the academics are the same.
queryB1 = """
                SELECT academic1.academic_name, academic2.academic_name
                FROM academic academic1
                JOIN editor_proceeding editor_proceeding1 ON academic1.academic_id = editor_proceeding1.academic_id
                JOIN proceeding ON editor_proceeding1.proceeding_id = proceeding.proceeding_id
                JOIN editor_proceeding editor_proceeding2 ON proceeding.proceeding_id = editor_proceeding2.proceeding_id
                JOIN academic academic2 ON editor_proceeding2.academic_id = academic2.academic_id
                WHERE academic1.academic_id <> academic2.academic_id
        """

duckdb.query(queryB1).to_df()

# B1 = cursor.execute(queryB1).fetchdf()
# B1

,academic_name,author_frequency
0,Maurizio Lenzerini,3
1,Leonid Libkin,3
2,Josep Lluís Larriba-Pey,3
3,Yannis Velegrakis,2
4,George H. L. Fletcher,2
5,Xin Luna Dong,2
6,Georgia Koutrika,2
7,Mirek Riedewald,2
8,Akhil Arora 0001,2
9,Arnab Bhattacharya 0001,2


In [ ]:
queryB2 = """
            """

,number_of_different_conferences_published
0,90


In [5]:
# Close the cursor and connection
duckdb_deps.close_duckdb(conn, cursor)
